In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [9]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [15]:
# cast(intime as date)

In [14]:
query = \
"""
WITH post AS (
    SELECT 
         ef.subject_id AS subject_id
        ,ef.icustay_id AS icustay_id
        ,ef.ed_charttime AS icu_charttime
        ,ef.ea_lv_systolic AS icu_lv_systolic
        ,ed.charttime AS post_charttime
    FROM echo_filtered ef
    INNER JOIN echodata ed
        ON ef.after_rowid = ed.row_id
    WHERE ef.after_rowid IS NOT null
)
SELECT pt.*
    ,ea2.new_time AS icu_matchtime
    ,ea2.lv_systolic AS icu_lv_systolic_match
    ,ea1.new_time AS post_matchttime
    ,ea1.lv_systolic AS post_lv_systolic
FROM post pt
LEFT JOIN echo_annotations_unique ea1
    ON ea1.icustay_id = pt.icustay_id
    AND ea1.new_time IS NOT DISTINCT FROM pt.post_charttime
LEFT JOIN echo_annotations_unique ea2
    ON ea2.icustay_id = pt.icustay_id
    AND ea2.new_time IS NOT DISTINCT FROM pt.icu_charttime
"""
eo = pd.read_sql_query(query, con)
eo.head()

,subject_id,icustay_id,icu_charttime,icu_lv_systolic,post_charttime,icu_matchtime,icu_lv_systolic_match,post_matchttime,post_lv_systolic
0,238,208413,2198-06-28 09:39:00,0.0,2198-09-06 10:30:00,2198-06-28 09:39:00,0.0,None,None
1,402,203761,2155-05-23 11:15:00,0.0,2156-09-06 13:00:00,2155-05-23 11:15:00,0.0,None,None
2,993,281702,2101-01-10 10:30:00,1.0,2101-02-09 13:00:00,2101-01-10 10:30:00,1.0,None,None
3,995,286486,2196-04-14 09:24:00,1.0,2197-01-18 10:00:00,2196-04-14 09:24:00,1.0,None,None
4,1006,232914,2158-10-24 15:16:00,NaN,2159-03-19 10:00:00,2158-10-24 15:16:00,NaN,None,None


None of the outpatient echos that we are interested in are annotated. 

Check echo reports to see if these echos show up there. 

In [12]:
eo.shape

(224, 9)

In [13]:
eo.dropna()

,subject_id,icustay_id,icu_charttime,icu_lv_systolic,post_charttime,icu_matchtime,icu_lv_systolic_match,post_matchttime,post_lv_systolic


In [67]:
query = \
"""
SELECT * FROM echo_annotations_unique
WHERE subject_id = 19334
"""
res = pd.read_sql_query(query, con)
res.sort_values('new_time')

,subject_id,hadm_id,age,gender,new_time,icustay_id,first_careunit,intime,outtime,pulmhtn,...,tv_pulm_htn,tv_tr,lv_cavity,lv_diastolic,lv_systolic,lv_wall,rv_cavity,rv_diastolic_fluid,rv_systolic,rv_wall
0,19334,155858,60.0,F,2101-12-13 09:45:00,241758,MICU,2102-01-02 09:43:00,2102-01-27 16:32:00,0,...,NaN,0.0,0,None,0,0.0,0.0,None,0.0,NaN
1,19334,155858,60.0,F,2102-01-03 11:11:00,241758,MICU,2102-01-02 09:43:00,2102-01-27 16:32:00,0,...,NaN,0.0,0,None,0,0.0,0.0,None,0.0,NaN
2,19334,155858,60.0,F,2102-01-10 14:57:00,241758,MICU,2102-01-02 09:43:00,2102-01-27 16:32:00,0,...,NaN,0.0,0,None,0,0.0,NaN,None,NaN,NaN
3,19334,183582,61.0,F,2102-12-26 09:00:00,275392,MICU,2103-01-08 18:39:00,2103-01-13 20:55:00,0,...,NaN,NaN,0,None,-1,1.0,0.0,None,0.0,NaN
4,19334,183582,61.0,F,2103-01-09 11:27:00,275392,MICU,2103-01-08 18:39:00,2103-01-13 20:55:00,0,...,NaN,NaN,0,None,1,0.0,0.0,None,0.0,0.0
5,19334,183582,61.0,F,2103-01-12 09:46:00,275392,MICU,2103-01-08 18:39:00,2103-01-13 20:55:00,0,...,NaN,NaN,0,None,1,NaN,0.0,None,0.0,NaN
6,19334,183582,61.0,F,2103-01-25 16:44:00,275392,MICU,2103-01-08 18:39:00,2103-01-13 20:55:00,0,...,1.0,1.0,0,None,0,NaN,0.0,None,0.0,NaN


In [68]:
query = \
"""
SELECT * FROM echodata
WHERE subject_id = 19334
"""
res = pd.read_sql_query(query, con)
res.sort_values('charttime')

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality
8,76455,19334,155858.0,2101-12-13,2101-12-13 09:45:00,Baseline. Pre- Chemotherapy.,58.0,154.0,1.63,126/64,126.0,64.0,NaN,Inpatient,TTE (Complete),Complete pulse and color flow,None,Adequate
1,76477,19334,155858.0,2102-01-03,2102-01-03 11:11:00,Left ventricular function.,64.0,160.0,1.78,123/70,123.0,70.0,94.0,Inpatient,Portable TTE (Complete),Complete pulse and color flow,None,Adequate
7,76454,19334,155858.0,2102-01-10,2102-01-10 14:57:00,Rule out endocarditis.,NaN,NaN,NaN,112/84,112.0,84.0,96.0,Inpatient,Portable TEE (Complete),Complete pulse and color flow,None,Adequate
6,76453,19334,183582.0,2102-12-26,2102-12-26 09:00:00,Baseline pre-CHEMO,58.0,148.0,1.60,None,NaN,NaN,78.0,Outpatient,TTE (Complete),Full doppler and color doppler,None,Adequate
5,76452,19334,183582.0,2103-01-09,2103-01-09 11:27:00,Left ventricular function.,58.0,148.0,1.60,83/40,83.0,40.0,83.0,Inpatient,Portable TTE (Focused views),No doppler,None,Adequate
4,76433,19334,183582.0,2103-01-12,2103-01-12 09:46:00,Left ventricular function.,58.0,148.0,1.60,102/53,102.0,53.0,89.0,Inpatient,Portable TTE (Focused views),No doppler,None,Adequate
3,76432,19334,183582.0,2103-01-25,2103-01-25 16:44:00,Chemotherapy. Left ventricular function.,59.0,180.0,1.76,116/68,116.0,68.0,81.0,Inpatient,Portable TTE (Complete),Full doppler and color doppler,None,Suboptimal
2,76431,19334,NaN,2103-09-11,2103-09-11 14:00:00,pre bone marrow transplant,59.0,150.0,1.63,100/46,100.0,46.0,95.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal
0,76412,19334,NaN,2103-10-19,2103-10-19 16:18:00,Endocarditis.,59.0,148.0,1.62,142/82,142.0,82.0,93.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate


In [70]:
query = \
"""
SELECT * FROM echo_annotations_unique
"""
ea = pd.read_sql_query(query, con)
ea['status'].value_counts()

Inpatient     31608
Outpatient     1069
Name: status, dtype: int64

In [71]:
query = \
"""
SELECT * FROM echodata
"""
ed = pd.read_sql_query(query, con)
ed['status'].value_counts()

Inpatient     39075
Outpatient     6719
Name: status, dtype: int64